The aim of this notebook is to illustrate the usage of *wnpoly*.  Begin by installing the necessary packages.

In [1]:
import sys, subprocess, pkg_resources
required = {'numpy', 'wnpoly'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    subprocess.check_call([sys.executable,'-m','pip','install','--quiet',*missing])

Next import the packages.

In [2]:
import numpy as np
import wnpoly as wp

*wnpoly* currently has two modules.  The first handles [Symmetric polynomials](#symmetric), which are polynomials that remain unchanged under exchange of any variables.  This [Wikipedia page](https://en.wikipedia.org/wiki/Symmetric_polynomial) provides an introduction.  The second handles [Bell polynomials](#bell), which appear particularly in problems involving [set partitions](https://en.wikipedia.org/wiki/Bell_polynomials).

<a id='symmetric'></a>
# Symmetric polynomials

This section illustrates *wnpoly's* symmetric polynomial classes, which include [complete homogeneous symmetric](https://en.wikipedia.org/wiki/Complete_homogeneous_symmetric_polynomial), [elementary homogeneous symmetric](https://en.wikipedia.org/wiki/Elementary_symmetric_polynomial), and [power sum symmetric](https://en.wikipedia.org/wiki/Power_sum_symmetric_polynomial) polynomials.

Begin by creating instances of each class.

In [3]:
chsp = wp.symm.complete()
ehsp = wp.symm.elementary()
pssp = wp.symm.power_sum()

Now create a set of variables *x*.  Here we choose a set of ten randomly chosen integers between 1 and 10.  By definition, *x[0]* should equal 0, so we insert that at the beginning of the array; thus, the array has 11 elements with index ranging from 0 to 10.  In fact, the *wnpoly* routines do not use *x[0]* in computations but rather use it as a placeholder to allow the index of the array to correspond to the expected *x* or polynomial.

In [4]:
n = 10
x = np.random.default_rng().integers(1, high=10, size=n)

Now compute the corresponding set of polynomials of the various types up to order *n*.

In [5]:
h = chsp.compute(x, n)
e = ehsp.compute(x, n)
p = pssp.compute(x, n)

Print out the values and the *x*'s.  Insert an extra zero at the beginning of the *x* array to align with the polynomials in the printout.

In [6]:
x = np.insert(x, 0, 0)

for n in range(len(x)):
    print("n = {:d}, x[n] = {:}, h[n] = {:d}, e[n] = {:d}, p[n] = {:d}".format(n, x[n], h[n], e[n], p[n]))

n = 0, x[n] = 0, h[n] = 1, e[n] = 1, p[n] = 1
n = 1, x[n] = 8, h[n] = 42, e[n] = 42, p[n] = 42
n = 2, x[n] = 2, h[n] = 1005, e[n] = 759, p[n] = 246
n = 3, x[n] = 1, h[n] = 18068, e[n] = 7736, p[n] = 1662
n = 4, x[n] = 3, h[n] = 271962, e[n] = 49011, p[n] = 11958
n = 5, x[n] = 7, h[n] = 3625716, e[n] = 200706, p[n] = 88902
n = 6, x[n] = 2, h[n] = 44272910, e[n] = 535649, p[n] = 674046
n = 7, x[n] = 1, h[n] = 506039904, e[n] = 916572, p[n] = 5176542
n = 8, x[n] = 5, h[n] = 5495511663, e[n] = 959796, p[n] = 40107558
n = 9, x[n] = 8, h[n] = 57310164622, e[n] = 554560, p[n] = 312716022
n = 10, x[n] = 5, h[n] = 578444634435, e[n] = 134400, p[n] = 2449551246


We can now check some of [Newton's identities](https://en.wikipedia.org/wiki/Newton%27s_identities).  The first we will check relates the complete and elementary polynomials such that $d_n = \sum_{k = 0}^n (-1)^k h_{n-k}(x_1, ..., x_n) e_k(x_1, ..., x_n) = 0$.

In [7]:
for n in range(1, len(x)):
    d = 0
    for k in range(n+1):
        d += np.power(-1, k) * h[n-k] * e[k]
    print("n = {:d}, d[n] = {:d}".format(n, d))

n = 1, d[n] = 0
n = 2, d[n] = 0
n = 3, d[n] = 0
n = 4, d[n] = 0
n = 5, d[n] = 0
n = 6, d[n] = 0
n = 7, d[n] = 0
n = 8, d[n] = 0
n = 9, d[n] = 0
n = 10, d[n] = 0


Now we check an identity relating the complete homogeneous symmetric and power sum symmetric polynomials, namely, that $nh_n = \sum_{k=1}^n h_{n-k} p_k$.

In [8]:
for n in range(1, len(x)):
    sum = 0
    for k in range(1, n+1):
        sum += h[n-k] * p[k]
    print(
        "n = {:d}, n*h[n] = {:d}, sum = {:d}, difference = {:d}".format(
            n, n*h[n], sum, n*h[n] - sum)
    )

n = 1, n*h[n] = 42, sum = 42, difference = 0
n = 2, n*h[n] = 2010, sum = 2010, difference = 0
n = 3, n*h[n] = 54204, sum = 54204, difference = 0
n = 4, n*h[n] = 1087848, sum = 1087848, difference = 0
n = 5, n*h[n] = 18128580, sum = 18128580, difference = 0
n = 6, n*h[n] = 265637460, sum = 265637460, difference = 0
n = 7, n*h[n] = 3542279328, sum = 3542279328, difference = 0
n = 8, n*h[n] = 43964093304, sum = 43964093304, difference = 0
n = 9, n*h[n] = 515791481598, sum = 515791481598, difference = 0
n = 10, n*h[n] = 5784446344350, sum = 5784446344350, difference = 0


Finally, we will check the identity that $n e_n = \sum_{k=1}^n (-1)^{k-1} e_{n-k} p_k$.

In [9]:
for n in range(1, len(x)):
    sum = 0
    for k in range(1, n+1):
        sum += np.power(-1, k-1) * e[n-k] * p[k]
    print(
        "n = {:d}, n*e[n] = {:d}, sum = {:d}, difference = {:d}".format(
            n, n*e[n], sum, n*e[n] - sum)
    )

n = 1, n*e[n] = 42, sum = 42, difference = 0
n = 2, n*e[n] = 1518, sum = 1518, difference = 0
n = 3, n*e[n] = 23208, sum = 23208, difference = 0
n = 4, n*e[n] = 196044, sum = 196044, difference = 0
n = 5, n*e[n] = 1003530, sum = 1003530, difference = 0
n = 6, n*e[n] = 3213894, sum = 3213894, difference = 0
n = 7, n*e[n] = 6416004, sum = 6416004, difference = 0
n = 8, n*e[n] = 7678368, sum = 7678368, difference = 0
n = 9, n*e[n] = 4991040, sum = 4991040, difference = 0
n = 10, n*e[n] = 1344000, sum = 1344000, difference = 0


It is also possible to compute the symmetric polynomials normalized by the number of terms in each polynomial.  Here one should not use integers but rather floats or similar type.  

In [10]:
n = 5
x = np.random.default_rng().uniform(0.0, 10.0, size=n)

Now compute and print out the normalized polynomials.  Again insert a zero at the beginning of *x* to align the *x*'s with the polynomials in the printout.

In [11]:
h_norm = chsp.compute_normalized(x, n)
e_norm = ehsp.compute_normalized(x, n)
p_norm = pssp.compute_normalized(x, n)

x = np.insert(x, 0, 0.)

for n in range(len(x)):
    print(
        "n = {:d}, x[n] = {:.4f}, h_norm[n] = {:.4f}, e_norm[n] = {:.4f}, p_norm[n] = {:.4f}".format(
        n, x[n], h_norm[n], e_norm[n], p_norm[n]))

n = 0, x[n] = 0.0000, h_norm[n] = 1.0000, e_norm[n] = 1.0000, p_norm[n] = 1.0000
n = 1, x[n] = 5.9531, h_norm[n] = 6.3818, e_norm[n] = 6.3818, p_norm[n] = 6.3818
n = 2, x[n] = 3.5961, h_norm[n] = 41.2990, e_norm[n] = 39.8695, p_norm[n] = 44.1581
n = 3, x[n] = 5.6731, h_norm[n] = 270.8505, e_norm[n] = 243.4582, p_norm[n] = 325.3983
n = 4, x[n] = 9.0551, h_norm[n] = 1799.1265, e_norm[n] = 1449.9228, p_norm[n] = 2514.8021
n = 5, x[n] = 7.6315, h_norm[n] = 12097.5495, e_norm[n] = 8392.7195, p_norm[n] = 20143.5994


<a id='bell'></a>
# Bell polynomials

This section illustrates *wnpoly's* Bell polynomial module.  The module has a class for complete and partial Bell polynomials.

Begin by creating an instance of the Bell polynomial class.

In [12]:
my_bell = wp.bell.bell()

Now generate some Bell polynomials.  By definition, $B_0 = 1$, so insert that.

In [13]:
n = 5
b = np.random.default_rng().uniform(low=1, high=10, size=n)
b = np.insert(b, 0, 1.)

Now invert the Bell polynomials to find the *x's* that would give rise to those Bell polynomials.

In [14]:
x = my_bell.invert(b)

Compute the Bell polynomials from those *x's* and use them to compute the Bell polynomials.  Compare the computed polynomials to the original ones.

In [15]:
bc = my_bell.compute(x)

for n in range(len(x)):
    print(
        "n = {:d}, x[n] = {:.6e}, b[n] = {:.6e}, bc[n] = {:.6e}".format(
            n, x[n], b[n], bc[n]
        )
    )

n = 0, x[n] = 0.000000e+00, b[n] = 1.000000e+00, bc[n] = 1.000000e+00
n = 1, x[n] = 9.860670e+00, b[n] = 9.860670e+00, bc[n] = 9.860670e+00
n = 2, x[n] = -8.906676e+01, b[n] = 8.166051e+00, bc[n] = 8.166051e+00
n = 3, x[n] = 1.679197e+03, b[n] = 3.203982e+00, bc[n] = 3.203982e+00
n = 4, x[n] = -4.752252e+04, b[n] = 1.128738e+00, bc[n] = 1.128738e+00
n = 5, x[n] = 1.793274e+06, b[n] = 2.275789e+00, bc[n] = 2.275789e+00


Now we study the partial Bell polynomial class.  First create an instance.

In [16]:
my_partial_bell = wp.bell.partial_bell()

Now compute the partial Bell polynomials from the *x's* above.  The return is a two-dimensional array with the indices *n* and *k* giving $B_{n,k}(x_1, ..., x_{n-k+1})$.

In [17]:
pbc = my_partial_bell.compute(x)

Check the results by noting that the complete Bell polynomials $B_n$ are given by $B_n(x_1, ..., x_n) = \sum_{k=1}^n B_{n,k}(x_1, ..., x_{n-k+1})$.  We note that $B_{0,0} = 1$ but $B_{n,0} = 0$ for $n > 0$.

In [18]:
for n in range(len(x)):
    bn = 0
    for k in range(0, n+1):
        bn += pbc[n, k]
    print(
        "n = {:d}, x[n] = {:.6e}, bc[n] = {:.6e}, bn = {:.6e}".format(
            n, x[n], bc[n], bn
        )
    )

n = 0, x[n] = 0.000000e+00, bc[n] = 1.000000e+00, bn = 1.000000e+00
n = 1, x[n] = 9.860670e+00, bc[n] = 9.860670e+00, bn = 9.860670e+00
n = 2, x[n] = -8.906676e+01, bc[n] = 8.166051e+00, bn = 8.166051e+00
n = 3, x[n] = 1.679197e+03, bc[n] = 3.203982e+00, bn = 3.203982e+00
n = 4, x[n] = -4.752252e+04, bc[n] = 1.128738e+00, bn = 1.128738e+00
n = 5, x[n] = 1.793274e+06, bc[n] = 2.275789e+00, bn = 2.275789e+00


As a final exercise, we note that $x_n = B_{n,1}$.  Here we check that.

In [19]:
for n in range(len(x)):
    print("n = {:d}, x[n] = {:.6e}, pbc[n,1] = {:.6e}".format(n, x[n], pbc[n, 1]))

n = 0, x[n] = 0.000000e+00, pbc[n,1] = 0.000000e+00
n = 1, x[n] = 9.860670e+00, pbc[n,1] = 9.860670e+00
n = 2, x[n] = -8.906676e+01, pbc[n,1] = -8.906676e+01
n = 3, x[n] = 1.679197e+03, pbc[n,1] = 1.679197e+03
n = 4, x[n] = -4.752252e+04, pbc[n,1] = -4.752252e+04
n = 5, x[n] = 1.793274e+06, pbc[n,1] = 1.793274e+06
